From: http://spark.apache.org/docs/1.2.1/mllib-linear-methods.html

In [1]:
import findspark
findspark.init()
# from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from numpy import array

In [2]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

In [3]:
conf = (SparkConf()
         .setMaster("local")
         .setAppName("sample local mlib")
         .set("spark.executor.memory", "512M")
         .set("spark.cores.max", 4))
# sc.stop()
sc = SparkContext(conf = conf)

textFile will still read this as RDD, but only store it locally

In [4]:
data = sc.textFile("file:///home/hduser/sandbox/sample_svm_data.txt")
parsedData = data.map(parsePoint)

In [5]:
# Build the model
model = LogisticRegressionWithSGD.train(parsedData)

/opt/spark/python/pyspark/mllib/classification.py:313: UserWarning: Deprecated in 2.0.0. Use ml.classification.LogisticRegression or LogisticRegressionWithLBFGS.
  "Deprecated in 2.0.0. Use ml.classification.LogisticRegression or "


Possible warning message in Logs:
https://www.cloudera.com/documentation/enterprise/5-7-x/topics/spark_mllib.html
<pre>
[I 00:05:32.839 NotebookApp] Saving file at /Sample Local machine learning (spark).ipynb
18/03/29 00:05:33 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
18/03/29 00:06:23 WARN util.SizeEstimator: Failed to check whether UseCompressedOops is set; assuming yes
18/03/29 00:06:53 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
18/03/29 00:06:53 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
</pre>

In [6]:
from itertools import starmap

# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
# labelsAndPreds.take(5)[0][1]
count = 0


tempList = labelsAndPreds.take(labelsAndPreds.count())
for x in range(labelsAndPreds.count()):
    if(tempList[x][0] != tempList[x][1]):
        count = count + 1
print("Score: {}/{}".format(count, parsedData.count()))
print("Training error = " + str(count / float(parsedData.count())))
# trainErr = labelsAndPreds.filter(lambda v, p: (v != p)).count() / float(parsedData.count())
# print("Training Error = " + str(trainErr))

Score: 117/322
Training error = 0.36335403726708076
